# Install Required Libraries

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets einops

# Log in to WandB to track finetuning metrics
# Log in to Hugging Face

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install wandb
!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 25.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=6ba97f857b0dbe4b1e07a3be4563fb9b29198d19f992d6d23239d4a03fbeec19
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Load & Quantize the model

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"":0})
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Import IA3 Config

In [5]:
from peft import IA3Config, get_peft_model

config = IA3Config(
    peft_type="IA3",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
    feedforward_modules=["v_proj"],
)

model = get_peft_model(model, config)

In [ ]:
print(model)

# Load and preprocessing the dataset

In [7]:
from datasets import load_dataset

# Split the data into 10% test, 10% validation, 80% training
train_data = load_dataset("Hieu-Pham/cooking_squad_splitted", split='train')
val_data = load_dataset("Hieu-Pham/cooking_squad_splitted", split='validation')

def preprocess_dataset(dataset):

    grouped = []

    for row in dataset:
      answers = row['answers']
      input = '[INST] Question: ' + row['question'] + ' Context: ' + row['context'] + ' [/INST]\nAnswer: ' + answers['text']
      grouped.append(input)

    return grouped

input_column = preprocess_dataset(train_data)
train_data = train_data.add_column('Inputs', input_column)

train_data = train_data.map(
    lambda row: tokenizer(row["Inputs"], truncation=True),
    batched=True,
    remove_columns=train_data.column_names,
)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1793 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
input_column[0]

In [ ]:
# Preprocessing validation dataset
# Validation dataset
output_column = preprocess_dataset(val_data)
val_data = val_data.add_column('Outputs', output_column)

val_data = val_data.map(
    lambda row: tokenizer(row["Outputs"], truncation=True),
    batched=True,
    remove_columns=val_data.column_names,
)

# Training Configurations and Start Training

In [12]:
import transformers

training_args = transformers.TrainingArguments(
    output_dir="Llama2-7B-IA3-cooking-text-gen-prompting",
    evaluation_strategy="steps",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_steps=10,
    logging_steps=10,
    max_grad_norm=0.3,
    max_steps=250,
    warmup_ratio=0.03,
    fp16=True,
    optim="paged_adamw_8bit",
    push_to_hub=True,
)


In [13]:
# tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings
trainer.train()

wandb: Currently logged in as: hieupham. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,1.507900,1.520704
20,1.521800,1.499739
30,1.473000,1.482325
40,1.478000,1.466670
50,1.444400,1.452335
60,1.473400,1.438896
70,1.435400,1.426260
80,1.406500,1.414092
90,1.390800,1.402017
100,1.386900,1.390359


TrainOutput(global_step=250, training_loss=1.371774471282959, metrics={'train_runtime': 447.0538, 'train_samples_per_second': 8.947, 'train_steps_per_second': 0.559, 'total_flos': 4.604233875935232e+16, 'train_loss': 1.371774471282959, 'epoch': 2.23})